# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [10]:
# Imports here

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from torch import nn
from torch import optim
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import CrossEntropyLoss
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from collections import OrderedDict
from torch.optim import SGD
from matplotlib.ticker import FormatStrFormatter
from PIL import Image

import torch
import torchvision
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np






## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [11]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [12]:
# TODO: Define your transforms for the training, validation, and testing sets
data_tr_transforms = transforms.Compose([transforms.RandomRotation(30),
                                     transforms.RandomResizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.485, 0.456, 0.406), 
                                                          (0.229, 0.224, 0.225))])

data_va_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.CenterCrop(244),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

data_te_transforms = transforms.Compose ([transforms.Resize(255),
                                         transforms.CenterCrop(244),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
image_datasets_tr = datasets.ImageFolder(train_dir, transform=data_tr_transforms)
image_datasets_va = datasets.ImageFolder(valid_dir, transform=data_va_transforms)
image_datasets_te = datasets.ImageFolder(test_dir, transform=data_te_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders_tr = torch.utils.data.DataLoader(image_datasets_tr, batch_size=256, shuffle=True)
dataloaders_va = torch.utils.data.DataLoader(image_datasets_va, batch_size=256, shuffle=False)
dataloaders_te = torch.utils.data.DataLoader(image_datasets_te, batch_size=256, shuffle=False)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [13]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

<font color='red'>**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.</font>

In [14]:
# TODO: Build and train your network

# calling a pretrained model
model = models.densenet121(pretrained=True)


# frezing ImageNet models' features parameters
for p in model.parameters():
    p.requires_grad=False


# Defining an new but untrained classifier model
classifier = nn.Sequential(OrderedDict([
                                 ('fcl1', nn.Linear(1024, 512)),
                                 ('relu1', nn.ReLU()),
                                 ('drop1', nn.Dropout(0.2)),
                                 ('fcl2', nn.Linear(512, 256)),
                                 ('relu2', nn.ReLU()),
                                 ('fcl3', nn.Linear(256, 102)),
                                 ('output', nn.LogSoftmax(dim=1))]))
                                  
model.classifier = classifier

classifier




/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Sequential(
  (fcl1): Linear(in_features=1024, out_features=512, bias=True)
  (relu1): ReLU()
  (drop1): Dropout(p=0.2)
  (fcl2): Linear(in_features=512, out_features=256, bias=True)
  (relu2): ReLU()
  (fcl3): Linear(in_features=256, out_features=102, bias=True)
  (output): LogSoftmax()
)

In [15]:
for device in ['cpu', 'cuda']:
    
    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
    # optimizer = optim.SGD(model.classifier.parameters(), lr=0.003, momentum=0.9)
    model.to(device)



    epochs = 3
    steps = 0
    training_loss = 0
    print_every = 5
    for epoch in range(epochs):
        for inputs, labels in dataloaders_tr:
            steps += 1
            # Move data and label tensors to the default device
            data, labels = inputs.to(device), labels.to(device)
        
            yhat1 = model.forward(data)
            loss = criterion(yhat1, labels)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            training_loss += loss.item()
        
            if steps % print_every == 0:
                validating_loss = 0
                accuracy = 0
                model.eval()
                with torch.no_grad():
                    for data, labels in dataloaders_va:
                        data, labels = data.to(device), labels.to(device)
                        yhat2 = model.forward(data)
                        batch_loss = criterion(yhat2, labels)
                    
                        validating_loss += batch_loss.item()
                    
                        # Calculate accuracy
                        ps = torch.exp(yhat2)
                        top_p, top_class = ps.topk(1, dim=1)
                        equals = top_class == labels.view(*top_class.shape)
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {training_loss/print_every:.3f}.. "
                      f"Validation loss: {validating_loss/len(dataloaders_va):.3f}.. "
                      f"Validation accuracy: {accuracy/len(dataloaders_va):.3f}")
                training_loss = 0
                model.train()

Epoch 1/3.. Train loss: 4.527.. Validation loss: 4.368.. Validation accuracy: 0.057
Epoch 1/3.. Train loss: 4.288.. Validation loss: 3.945.. Validation accuracy: 0.137
Epoch 1/3.. Train loss: 3.813.. Validation loss: 3.443.. Validation accuracy: 0.207
Epoch 1/3.. Train loss: 3.250.. Validation loss: 2.646.. Validation accuracy: 0.392
Epoch 1/3.. Train loss: 2.658.. Validation loss: 2.009.. Validation accuracy: 0.513
Epoch 2/3.. Train loss: 2.162.. Validation loss: 1.763.. Validation accuracy: 0.512
Epoch 2/3.. Train loss: 1.858.. Validation loss: 1.496.. Validation accuracy: 0.573
Epoch 2/3.. Train loss: 1.708.. Validation loss: 1.148.. Validation accuracy: 0.689
Epoch 2/3.. Train loss: 1.525.. Validation loss: 1.193.. Validation accuracy: 0.673
Epoch 2/3.. Train loss: 1.388.. Validation loss: 0.932.. Validation accuracy: 0.749
Epoch 3/3.. Train loss: 1.275.. Validation loss: 0.855.. Validation accuracy: 0.767
Epoch 3/3.. Train loss: 1.155.. Validation loss: 0.843.. Validation accuracy

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [22]:
# testing our model on testset
epochs = 1
steps = 0
print_every = 5

for epoch in range(epochs):

    if steps % print_every == 0:
        testing_loss = 0
        accuracy = 0
        model.eval()
        with torch.no_grad():
            for data, labels in dataloaders_te:
                data, labels = data.to(device), labels.to(device)
                yhat3 = model.forward(data)
                batch_loss = criterion(yhat3, labels)
                    
                testing_loss += batch_loss.item()
                    
                # Calculate accuracy
                ps = torch.exp(yhat3)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Testing loss: {testing_loss/len(dataloaders_te):.3f}.. "
                      f"Testset accuracy: {accuracy/len(dataloaders_te):.3f}")
               

Epoch 1/1.. Testing loss: 0.129.. Testset accuracy: 0.212
Epoch 1/1.. Testing loss: 0.217.. Testset accuracy: 0.428
Epoch 1/1.. Testing loss: 0.306.. Testset accuracy: 0.646
Epoch 1/1.. Testing loss: 0.520.. Testset accuracy: 0.828


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [34]:

# TODO: Save the checkpoint

class_to_idx = image_datasets_tr.class_to_idx
model.class_to_idx = { class_to_idx[k]: k for k in class_to_idx}

checkpoint4 = {'arch': 'densenet121',
              'batch_size': 256,
              'epochs': 3,
              'criterion': 'nn.NLLLoss()',
              'learning_rate': 0.003, 
              'print_every': 5,
              'best_test_accuracy': 0.895,
              'classifier': model.classifier,
              'optimizer_dict': optimizer.state_dict(),
              'class_to_idx': model.class_to_idx,
              'state_dict': model.state_dict(),
              
              }
              
                
torch.save(checkpoint4, 'checkpoint.pth')
print("Checkpoint saved!!!")

Checkpoint saved!!!


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [35]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
   
def load_model(filepath):
    
    '''This function should take the checkpoint file path, load the checkpoint and
    extract the model information and return that fantastic model that was created earlier'''
    
    checkpoint = torch.load(filepath)
    model = models.densenet121(pretrained=True)
    model.arch = checkpoint['arch']
    model.class_to_idx = checkpoint['class_to_idx']
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_dict'])
    
            
    return model


model = load_model('checkpoint.pth')
print(model)
print(checkpoint1['criterion'])

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [36]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    with Image.open(image) as im:
        
        transformer = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(244),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        
        x_im = transformer(im)
    
        np_image = np.array(x_im)
           
    return np.np_image.T

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [37]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    
    return probs, classes

probs, classes = predict(image_path, model)
print(probs)
print(classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

In [ ]:
# 

<font color='red'>**Reminder for Workspace users:** If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again. 
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.</font>

In [ ]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace